In [2]:
# from __future__ import absolute_import, division, prision, prin_fuction, unicode_literals
import tensorflow as tf


In [1]:
!pip install -q imageio

In [4]:
import glob, imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
from IPython import display

In [16]:
(train_images, train_labels),(_,_) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0],28,28,1).astype('float32')
train_images = (train_images - 127.5) / 127.5
train_images[0].shape

(28, 28, 1)

In [17]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256


In [18]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Reshape((7,7,256)))
    assert model.ouput_shape == (None, 7,7,256)
    
    model.add(layers.Conv2DTranspose(128, (5,5), strides=(1,1), padding='same', use_bias=False)
              assert model.output